# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name: Otabek Azimov

Student ID: 16BD02134

Email: a.habibullayevich@gmail.com

In [3]:
import numpy as np
import pandas as pd
import sklearn.neighbors
import sklearn.ensemble as ensemble
import math
import matplotlib.pyplot as plt
%matplotlib inline

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [4]:
data = pd.read_csv("2008.csv.bz2")
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7.009728e+06,6.855029e+06,...,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,2.224200e+03,1.273224e+02,...,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,1.961716e+03,7.018731e+01,...,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,...,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.220000e+02,7.700000e+01,...,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,1.571000e+03,1.100000e+02,...,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,3.518000e+03,1.570000e+02,...,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01
max,2008.0,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.743000e+03,1.379000e+03,...,4.962000e+03,3.080000e+02,4.290000e+02,1.000000e+00,1.000000e+00,2.436000e+03,1.352000e+03,1.357000e+03,3.920000e+02,1.316000e+03


In [5]:
1.Какая из причин отмены рейса (CancellationCode) была самой частой? (расшифровки кодов можно найти в описании данных)

SyntaxError: invalid syntax (<ipython-input-5-89395df2859a>, line 1)

In [ ]:
DF = pd.read_csv("2008.csv.bz2")
print(DF.head())

In [ ]:
CC = DF['CancellationCode']
CC.value_counts().plot(kind='bar')

In [ ]:
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.

In [ ]:
print("Max: ", DF.Distance.max())
print("Min: ", DF.Distance.min())
print("Mean: ", DF.Distance.mean())

In [ ]:
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?

In [ ]:
DF[DF.Distance == DF.Distance.min()]


In [ ]:
DF[DF.FlightNum == 4988]['Distance'].values

In [ ]:
DF[DF.FlightNum == 5572]['Distance'].values

In [ ]:
Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?

In [ ]:
DF["Origin"].value_counts().index[0]

In [ ]:
Найдите для каждого аэропорта среднее время полета (AirTime) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?

In [ ]:
DF.groupby("Origin")['AirTime'].mean()

In [ ]:
print("Max mean air time: ", DF.groupby("Origin")['AirTime'].mean().argmax())

In [ ]:
Найдите аэропорт, у которого наибольшая доля задержанных (DepDelay > 0) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию filter после groupby).

In [ ]:
DFGB = DF.groupby("Origin")['DepDelay']
DFGB_Filtered = DF[DF.DepDelay > 0].groupby("Origin")['DepDelay']
(DFGB_Filtered.count() / DFGB.count() * (DFGB.count() >= 1000)).argmax()

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [ ]:
data = pd.read_csv('amazon.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# доля положительных примеров
data.ACTION.mean()

In [ ]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [ ]:
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

In [ ]:
Hash_sid = np.empty((9), dtype=dict)
Hash_id = np.empty((9), dtype=dict)
L = len(X_train)
for i, feature in enumerate(X_train.columns):
    Hash_sid[i] = {}
    Hash_id[i] = {}
F = X_train[feature].value_counts()
feature_values = X_train[feature].unique()
for x in feature_values:
    Hash_sid[i][x] = F[x] * (F[x] - 1) / (L * (L - 1))
    Hash_id[i][x] = math.log(F[x] + 1)

In [ ]:
def indicator_dist(x, y):
    return (x != y).sum()
def smoothy_indicator_dist(x, y):
    return np.array([(x[c] != y[c]) + (x[c] == y[c]) * Hash_sid[c].get(x[c], 0) for c in range(9)]).sum()
def importance_dist(x, y):
    return np.array([(x[c] != y[c]) * Hash_id[c].get(x[c], 0) * Hash_id[c].get(y[c], 0) for c in range(9)]).sum()

In [ ]:
Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm="brute", metric=indicator_dist).fit(X_train, y_train)
print(sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(X_test)[:, 1]))

In [ ]:
Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm="brute", metric=smoothy_indicator_dist).fit(X_train, y_train)
print(sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(X_test)[:, 1]))

In [ ]:
Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm="brute", metric=importance_dist).fit(X_train, y_train)
print(sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(X_test)[:, 1]))

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

In [ ]:
def MakeCounters(train, extractable_data, extractable_data_ans):
        train_modified = pd.DataFrame(index=train.index)
        for feature in train.columns:
            vc = extractable_data[feature].value_counts()
            vc_t = (extractable_data[feature] * extractable_data_ans).value_counts()
            train_modified[feature + "_counts"] = pd.Series([vc.get(x, 0) for x in train[feature]], index=train.index),
            train_modified[feature + "_successes"] = pd.Series([vc_t.get(x, 0) for x in train[feature]], index=train.index)
    train_modified[feature + "_cs"] = (train_modified[feature + "_successes"] + 1) / (train_modified[feature + "_counts"] + 2)
        return train_modified"

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [ ]:
mod_train = MakeCounters(X_train, X_train, y_train)
mod_test = MakeCounters(X_test, X_train, y_train)
k_list = np.arange(1, 20)
auc_roc = np.empty(k_list.shape)
for i, k in enumerate(k_list):
    Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k, algorithm="kd_tree").fit(mod_train, y_train)
    auc_roc[i] = sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(mod_test)[:, 1])
plt.plot(k_list, auc_roc)
plt.show()
print("BEST k: ", k_list[np.argmax(auc_roc)])
print("BEST AUC-ROC: ", np.max(auc_roc))

In [ ]:
L = len(X_train)
mt1 = MakeCounters(X_train[:L//3], X_train[L//3:], y_train[L//3:])
indexes = np.append(np.arange(0, L//3), np.arange(2*L//3, L))
mt2 = MakeCounters(X_train[L//3:2*L//3], X_train.iloc[indexes], y_train.iloc[indexes])
mt3 = MakeCounters(X_train[2*L//3:], X_train[:2*L//3], y_train[:2*L//3])
mt = pd.concat([mt1, mt2, mt3])
for feature in mt.columns[np.arange(0, len(mt.columns), 3)]:
     mt[feature] *= 3 / 2
for feature in mt.columns[np.arange(1, len(mt.columns), 3)]:
    mt[feature] *= 3 / 2\
    return mt

In [ ]:
mod_train = MakeCountersWithFolding(X_train, y_train)
mod_test = MakeCounters(X_test, X_train, y_train)
k_list = np.arange(1, 25)
auc_roc = np.empty(k_list.shape)
for i, k in enumerate(k_list):
    Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k, algorithm=\"kd_tree\").fit(mod_train, y_train)
    auc_roc[i] = sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(mod_test)[:, 1])
plt.plot(k_list, auc_roc)
plt.show()
print("BEST k: ", k_list[np.argmax(auc_roc)])
print("BEST AUC-ROC: ", np.max(auc_roc))                                                      

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [ ]:
def MakePairFeatures(train):
train_modified = train.copy()
for i, feature1 in enumerate(train.columns):
    for feature2 in train.columns[i+1:]:
    train_modified[feature1 + "+" + feature2] = train[feature1].astype(str) + "#" + train[feature2].astype(str)
return train_modified

In [ ]:
train, test = MakePairFeatures(X_train), MakePairFeatures(X_test)
mod_train_wf = MakeCountersWithFolding(train, y_train)
mod_test_wf = MakeCounters(test, train, y_train)
k_list = np.arange(5, 25)
auc_roc = np.empty(k_list.shape)
for i, k in enumerate(k_list):
    Classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k, algorithm="kd_tree", n_jobs = -1).fit(mod_train_wf, y_train)
    auc_roc[i] = sklearn.metrics.roc_auc_score(y_test, Classifier.predict_proba(mod_test_wf)[:, 1])
plt.plot(k_list, auc_roc)
plt.show()
print("BEST k: ", k_list[np.argmax(auc_roc)])
print("BEST AUC-ROC: ", np.max(auc_roc))

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [ ]:
depth = np.arange(4, 30)
samples_leaf = np.arange(8, 20).astype(int)
xgrid, ygrid = np.meshgrid(depth, samples_leaf)
zgrid = np.zeros(xgrid.shape)
for sl in range(len(samples_leaf)):
    for d in range(len(depth)):
        for test_k in range(5):
            clf = tree.DecisionTreeClassifier(max_depth=depth[d], min_samples_leaf=samples_leaf[sl])
            clf = clf.fit(mod_train, y_train)
            zgrid[sl][d] += sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test)[:, 1])
            zgrid[sl][d] /= 5
plt.pcolor(xgrid, ygrid, zgrid)
plt.show()

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=26, min_samples_leaf=13)
clf = clf.fit(mod_train, y_train)
print(sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test)[:, 1]))

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

In [ ]:
n_set = np.arange(1, 20)
results = np.zeros(n_set.shape[0])
overfitting = np.zeros(n_set.shape[0])
for i, n in enumerate(n_set):
    for k in range(5):
        clf = ensemble.RandomForestClassifier(n_estimators = n, max_depth=8, min_samples_leaf=15, n_jobs = -1)
        clf = clf.fit(mod_train_wf, y_train)
        results[i] += sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test_wf)[:, 1])
        overfitting[i] += sklearn.metrics.roc_auc_score(y_train, clf.predict_proba(mod_train_wf)[:, 1])
        results[i] /= 5
        overfitting[i] /= 5
plt.plot(n_set, results)
plt.plot(n_set, overfitting)
plt.show()

In [ ]:
clf = ensemble.RandomForestClassifier(n_estimators = 16, max_depth=8, min_samples_leaf=15, n_jobs = -1)
clf = clf.fit(mod_train_wf, y_train)
print("FINAL RESULT: ", sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(mod_test_wf)[:, 1]))

Здесь вы можете поделиться своими мыслями о задании.

In [ ]:
Было сложно. Думаю нужно больше практики. И теоритического материала, который поможет изучать больше самостоятельно
и не отстовать от лектора.

А здесь вставьте смешную картинку.

In [ ]:
https://i.redd.it/fnyz8lt1e8o11.jpg

А здесь посоветуйте преподавателям хороший фильм или сериал.

In [ ]:
Всё ещё Элис